In [ ]:
using Flux
using Flux.Tracker
using Flux: argmax, crossentropy,throttle,onehotbatch
using Base.Iterators: repeated, partition
using Plotly
using DataFrames
using CategoricalArrays

In [ ]:
xor(true,true),xor(false,true),xor(false,false),xor(true,false)

In [ ]:
xor(1,1),xor(0,1),xor(0,0),xor(1,0)

In [ ]:
a=rand([0,1],1_00);
b=rand([0,1],1_00);
X=2.* ([a b] .- 0.5)'
Y = 2.*(xor.(a,b) .- 0.5);
Y2 = onehotbatch([Y...],[-1.0,1.0]);

In [ ]:
w1 = param(randn(4,2))
b1 = param(randn(4))
layer1(x) = w1 * x .+ b1

w2 = param(rand(2,4))
b2 = param(rand(2))
layer2(x) = w2 * x .+b2

model2(x) = softmax(layer2(relu.(layer1(x))))

In [ ]:
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)

In [ ]:
#loss1(x,y) = meansquarederror(model(x),y)
loss2(x,y) = crossentropy(model2(x),y)

In [ ]:
function update!(ps,η=.1)
    for w in ps
        w.data .-=w.grad .* η
        w.grad .=0
    end
end

In [ ]:
accuracym2(x, y) = mean(argmax(model2(x)) .== argmax(y))
for i = 1:100
    back!(loss2(X,Y2))
    update!((w1,b1,w2,b2))
    @show accuracym2(X,Y2)
end

In [ ]:
a=rand([0,1],1_000);
b=rand([0,1],1_000);
testX=2.* ([a b] .- 0.5)'
testY = 2.*(xor.(a,b) .- 0.5);
testY2 = onehotbatch([testY...],[-1.0,1.0]);
accuracym2(testX,testY2)

In [ ]:
w1.data

In [ ]:
sample_size = 10_000
a = rand([-3:0.01:3]...,sample_size)
b = rand([-3:0.01:3]...,sample_size)
data= DataFrame(axisx = a, axisy = b)
data[:class] = CategoricalArrays.CategoricalArray{String,1,UInt8}(sample_size)
for r in eachrow(data)
    pred = model2([r[:axisx];r[:axisy]])
    if pred.data[1] == pred.data[2]
        r[:class] = "Non"
        continue
    end
    if argmax(pred) == 1
        r[:class] = "F"
    else
        r[:class] = "T"
    end
end

p_model_lowlevel = Plot(data, x=:axisx, y=:axisy, mode="markers", marker_size=8, group=:class)

plot(p_model_lowlevel)

In [ ]:
a=rand([0,1],100);
b=rand([0,1],100);
X=2.* ([a b] .- 0.5)'
Y = 2.*(xor.(a,b) .- 0.5);
Y2 = onehotbatch([Y...],[-1.0,1.0]);
dataset = repeated((X,Y2),5_000);

m = Chain(
  Dense(2, 4, relu),
  Dense(4, 2),softmax)

# Model loss function
loss(x, y) = crossentropy(m(x), y)
opt = ADAM(params(m))
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))
evalcb = () -> @show(accuracy(X, Y2))
# later
Flux.train!(loss, dataset, opt,cb = throttle(evalcb,1))

In [ ]:
sample_size = 10_000
a = rand([-3:0.01:3]...,sample_size)
b = rand([-3:0.01:3]...,sample_size)
data= DataFrame(axisx = a, axisy = b)
data[:class] = CategoricalArrays.CategoricalArray{String,1,UInt8}(sample_size)
for r in eachrow(data)
    pred = m([r[:axisx];r[:axisy]])
    if pred.data[1] == pred.data[2]
        r[:class] = "Non"
        continue
    end
    if argmax(pred) == 1
        r[:class] = "F"
    else
        r[:class] = "T"
    end
end

p_model_highlevel = Plot(data, x=:axisx, y=:axisy, mode="markers", marker_size=8, group=:class)

plot(p_model_highlevel)

In [ ]:
layer1_θ = m.layers[1]
layer2_θ = m.layers[2]

In [ ]:
layer1_θ.b.data

In [ ]:
layer1_θ.W.data = [-1 1;1 1;1 -1;-1 -1]

In [ ]:
layer1_θ.b.data = [-1; -1; -1; -1]

In [ ]:
layer2_θ.W.data = [0 1 0 1;1 0 1 0]

In [ ]:
layer2_θ.b.data = [0;0]

In [ ]:
sample_size = 10_000
a = rand([-3:0.01:3]...,sample_size)
b = rand([-3:0.01:3]...,sample_size)
data= DataFrame(axisx = a, axisy = b)
data[:class] = CategoricalArrays.CategoricalArray{String,1,UInt8}(sample_size)
for r in eachrow(data)
    pred = m([r[:axisx];r[:axisy]])
    if pred.data[1] == pred.data[2]
        r[:class] = "Non"
        continue
    end
    if argmax(pred) == 1
        r[:class] = "F"
    else
        r[:class] = "T"
    end
end

p_manual = Plot(data, x=:axisx, y=:axisy, mode="markers", marker_size=8, group=:class)

Plotly.plot(p_manual)

In [ ]:
a=rand([0,1],10);
b=rand([0,1],10);
X=2.* ([a b] .- 0.5)'
Y = 2.*(xor.(a,b) .- 0.5);
Y2 = onehotbatch([Y...],[-1.0,1.0]);
dataset = repeated((X,Y2),50_000);
m = Chain(
  Dense(2, 10,relu),
  Dense(10,8,relu),
  Dense(8,4,relu),
  Dense(4, 2),softmax)

# Model loss function
loss(x, y) = crossentropy(m(x), y)
opt = ADAM(params(m))
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))
evalcb = () -> @show(accuracy(X, Y2))
# later
Flux.train!(loss, dataset, opt,cb = throttle(evalcb,5))

In [ ]:
sample_size = 10_000
a = rand([-3:0.01:3]...,sample_size)
b = rand([-3:0.01:3]...,sample_size)
data= DataFrame(axisx = a, axisy = b)
data[:class] = CategoricalArrays.CategoricalArray{String,1,UInt8}(sample_size)
for r in eachrow(data)
    pred = m([r[:axisx];r[:axisy]])
    if pred.data[1] == pred.data[2]
        r[:class] = "Non"
        continue
    end
    if argmax(pred) == 1
        r[:class] = "F"
    else
        r[:class] = "T"
    end
end

p_morelayers = Plot(data, x=:axisx, y=:axisy, mode="markers", marker_size=3, group=:class)

plot(p_morelayers)